In [1]:
import numpy as np, scipy as sp, pandas as pd, sklearn as skl, matplotlib.pyplot as plt 
import xgboost as xgb
%matplotlib inline

In [2]:
# load data 
train_orig = pd.read_csv('data/train.csv')
test_orig = pd.read_csv('data/test.csv')
subm_orig = pd.read_csv('data/sample_submission.csv')

In [3]:
#ttypes = [int(x.split('_')[1]) for x in subm_orig.columns[1:]]
#int2ttype = dict([(i,x) for  i,x in enumerate(ttypes)])
#ttype2int = dict([(x,i) for  i,x in enumerate(ttypes)])
lencTT=skl.preprocessing.LabelEncoder()
lencTT.fit(train_orig.TripType.reshape(-1,1))

# do not use label encoder for weekdays, retain order 
d2int = {}
d2int['Monday'] = 2
d2int['Tuesday'] = 3
d2int['Wednesday'] = 4
d2int['Thursday'] = 5
d2int['Friday'] = 6
d2int['Saturday'] = 0
d2int['Sunday'] = 1


/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
# feature desc
#DD: can be nan, need to fill 
# scan count, can be negative (-12,71)


In [5]:
train=train_orig.copy()

lencDD = skl.preprocessing.LabelEncoder()
lencDD.fit(np.unique(train['DepartmentDescription'].fillna('N/A')))

def OHEncCol(df, col): # col must be numerical value 
    enc=skl.preprocessing.OneHotEncoder()
    enc.fit(df[col].reshape(-1,1))
    newCols = [col + '_' + str(x) for x in enc.active_features_]
    df=pd.concat([df, pd.DataFrame(columns=newCols)])
    df[newCols] = enc.transform(df[col].reshape(-1,1)).toarray()
    return df 

def preprocess(df, lencDD = lencDD):
    df['VisitNumber'] = df['VisitNumber'].astype(np.int32)
    if 'TripType' in df.columns:
        df['TTint']=lencTT.transform(df.TripType.reshape(-1,1)) # only do this w /training sample 
    df['Wdint']=df.Weekday.replace(d2int)
    df['FinelineNumber']=df.FinelineNumber.fillna(-1).astype(int)
    df['DepartmentDescription'] = df['DepartmentDescription'].fillna('N/A')
    df['fldd'] = df['FinelineNumber'].apply(lambda x:str(int(x)))+ '_' +df['DepartmentDescription'].astype(str)
    df['DDint'] = lencDD.transform(df['DepartmentDescription'])
    # OH encode weekdays 
    
    print (df.dtypes)
    return df


train = preprocess(train)
#train=OHEncCol(train, 'Wdint')


/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TripType                   int64
VisitNumber                int32
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber             int64
TTint                      int64
Wdint                      int64
fldd                      object
DDint                      int64
dtype: object


In [6]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(
#    #train[['VisitNumber','Wdint','Upc','ScanCount','DepartmentDescription','FinelineNumber']]
#    train[features]
#    ,train[['TTint']],test_size=0.3, random_state=10)

# X_train

# splitting using train_test_split doesn't work, need to split by visit number
uniqv = np.unique(train.VisitNumber) 
vn2 = set(np.random.choice(uniqv, int(len(uniqv)*0.3))) ; print len(uniqv), len(vn2)
idx1 = [x not in vn2 for x in train.VisitNumber]
idx2 = [x in vn2 for x in train.VisitNumber]
train1 = train[idx1]
train2 = train[idx2]

95674 24645


In [7]:
def multiclassLogLoss(y_true, y_pred, eps=1e-15):   # true is n x 1, pred is n x p, 
    predictions = np.clip(y_pred, eps, 1 - eps)
    # normalize
    predictions /= predictions.sum(axis=1)[:, np.newaxis]
    ohenc = skl.preprocessing.OneHotEncoder().fit(np.arange(y_pred.shape[1]).reshape(-1,1))
    actual = ohenc.transform(y_true).toarray() # convert y into matrix w/ OH enc 
    return -1.0 / predictions.shape[0] * np.sum(actual * np.log(predictions))



In [98]:
# EDA 
#(train.groupby(['FinelineNumber','DepartmentDescription'])['ScanCount'].agg(len)>10).sum()
# discretize <-1, -1, 1, 2, 3, 4-5, > 5
#train.groupby('VisitNumber')['TripType'].agg(lambda x:len(np.unique(x))) # can predict by VN, instead of by line 
# plt.hist(np.minimum(30, train.groupby('VisitNumber')['Upc'].agg(len).values))  # distri of items purchased 
df = train.copy()
minObs = 100  #  minObs 100 -> 1300 fldd, 500->200, 1000-> 69. , 
print (df.groupby(['fldd'])['ScanCount'].agg(len)>=minObs).sum()
foo = df.groupby('fldd')['fldd'].agg(len)
popularFldd = (foo[foo>minObs].index.values)

lenc_fldd = skl.preprocessing.LabelEncoder()
lenc_fldd.fit(np.array(list(popularFldd)) ) # .reshape(-1,1))

df['isPopular'] = df.groupby('fldd').apply(lambda x: pd.Series([len(x)>=minObs] * len(x), index=x.index)).values
print df.groupby('isPopular')['fldd'].agg(len)  # covers >80% of total obs 
#foo[foo]

1382
isPopular
False    183604
True     463450
Name: fldd, dtype: int64


In [99]:
# feature space, Wdint (7) + sum(scancount of fldd)

#df = train.copy() 
def makeSparseMat(df) : 
    dfVN = df[['VisitNumber','TripType','TTint','Wdint']].groupby(  # + ['Wdint_' + str(x) for x in xrange(7)]].groupby(
        'VisitNumber').agg(lambda x:x.iloc[0])  # takes one minute, don't run every time 
    # dfVN =OHEncCol(dfVN, 'Wdint') #  no need to do OHE here, moved to sparse 

    dfVN['VisitNumber'] = dfVN.index 
    M, N = len(dfVN)  , len(popularFldd) + 7 # + weekdays 

    lenc_pfldd = skl.preprocessing.LabelEncoder()
    lenc_pfldd.fit(popularFldd)
    lencClsSet = set(lenc_pfldd.classes_)

    scByVF = df.groupby(['VisitNumber','fldd'])['ScanCount'].agg(np.sum).reset_index()  # group by vn, fldd, and sum scancount 
    vnRowMap = pd.Series(np.arange(len(dfVN)), index=dfVN.index) # map from visit number to row index 
    csrDat = [0] * len(scByVF) * 2 
    csrRowi = [0] * len(scByVF) * 2 
    csrColi = [0] * len(scByVF) * 2  # times 2 to combine w/ weekday sparse mat as well 
    for i, r in dfVN.iterrows(): 
        csrDat[M + i] = 1 # need to fill weekday info 
        csrRowi[M + i] = vnRowMap.loc[r['VisitNumber']]
        csrColi[M + i] =r['Wdint']
    for i, r in scByVF.iterrows():
        if i%10000 == 0 : 
            print i
        if r['fldd'] in lencClsSet:    
            csrDat[i] = r['ScanCount']
            csrRowi[i] = vnRowMap.loc[r['VisitNumber']]
            csrColi[i] = lenc_pfldd.transform(np.array([r['fldd']]))

    sparMat = sp.sparse.csr_matrix((csrDat, (csrRowi, csrColi)), shape=(M,N))
    return sparMat , dfVN['TTint'].values.reshape(-1,1)

sp1, lb1 = makeSparseMat(train1) 
sp2, lb2 = makeSparseMat(train2)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [100]:

print len(vnRowMap), sparMat.shape
sparMat.toarray()[:30, :15]
sp1.shape, sp2.shape


95674 (95674, 76)


((71029, 1381), (24645, 1381))

In [9]:
# feature space, Wdint (7) + sum(scancount of fldd)

df = train.copy() 
dfVN = df[['VisitNumber','TripType','TTint','Wdint']].groupby(  # + ['Wdint_' + str(x) for x in xrange(7)]].groupby(
    'VisitNumber').agg(lambda x:x.iloc[0])  # takes one minute, don't run every time 
dfVN =OHEncCol(dfVN, 'Wdint')

# takes a long time to run  

for f in popularFldd:
    dfVN[f] = 0

#mat = np.zeros((len(dfVN), len(popularFldd)))
for i, r in df.iterrows():#len(df)):
    if i%10000 == 0 : 
        print i
    if r['fldd'] in popularFldd:
        dfVN.loc[r['VisitNumber'], r['fldd']] += r['ScanCount']
    else:
        continue
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000


In [10]:
features = ['Wdint_' + str(x) for x in xrange(7)] + list(popularFldd)
sub1 = [x not in vn2 for x in dfVN.index]
sub2 = [x in vn2 for x in dfVN.index]
dmtrain1 = xgb.DMatrix(dfVN.loc[sub1, features], label = dfVN.loc[sub1,'TTint'].tolist())
dmtrain2 = xgb.DMatrix(dfVN.loc[sub2, features], label = dfVN.loc[sub2,'TTint'].tolist())

In [101]:
# sparse version 
features = ['Wdint_' + str(x) for x in xrange(7)] + list(popularFldd)
dmtrain1 = xgb.DMatrix(sp1, label=lb1)
dmtrain2 = xgb.DMatrix(sp2, label=lb2)

In [102]:
#dmtrain1.save_binary('dmt1.buffer')
#dmtrain2.save_binary('dmt2.buffer')
#dmtrain1 = xgb.DMatrix('dmt1.buffer')
#dmtrain2 = xgb.DMatrix('dmt2.buffer')

In [109]:
%timeit 
xgb_params = {'max_depth': 10,
          'objective': 'multi:softprob',
          'eval_metric': 'mlogloss',
          'num_class': 38,
          'subsample': 0.35,
          'colsample_bytree': 1,
          'eta': 0.7}

watchlist = [(dmtrain1, 'train'), (dmtrain2, 'eval')]
mod = xgb.train(xgb_params, dmtrain1, evals = watchlist,  verbose_eval=False, early_stopping_rounds=25)
# mod.fit(X_train, y_train, eval_metric='mlogloss')

In [110]:
pred = mod.predict(dmtrain2)
multiclassLogLoss(dfVN.loc[sub2, ['TTint']].values, pred)   # 70 ftr, 2.8, 1300 ftr -> 2.1 

2.1618854733313904

In [24]:
# comparing non popular prediction vs modeled 
hasData = dfVN.loc[sub2, features].values[:,7:].sum(1) > 0
multiclassLogLoss(dfVN.loc[sub2, ['TTint']][hasData].values, pred[hasData]), multiclassLogLoss(dfVN.loc[sub2, ['TTint']][np.logical_not(hasData)].values, pred[np.logical_not(hasData)])

(2.509177072613519, 2.9781298680871839)

In [27]:
dfTest = test_orig.copy()
dfTest = preprocess(dfTest)

VisitNumber                int32
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber             int64
Wdint                      int64
fldd                      object
DDint                      int64
dtype: object


In [30]:
def aggByTrip(df, popularFldd):

    dfVN = df[[x for x in ['VisitNumber','TripType','TTint','Wdint'] if x in df.columns]].groupby( 
        'VisitNumber').agg(lambda x:x.iloc[0])  # takes one minute, don't run every time 
    dfVN =OHEncCol(dfVN, 'Wdint')

    for f in popularFldd:
        dfVN[f] = 0

    for i, r in df.iterrows():#len(df)):
        if i%10000 == 0 : 
            print i
        if r['fldd'] in popularFldd:
            dfVN.loc[r['VisitNumber'], r['fldd']] += r['ScanCount']
        else:
            continue
    return dfVN 

dftVN = aggByTrip(dfTest, popularFldd)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000


In [57]:
subMat =  np.concatenate((dftVN.index.values.reshape(-1,1), mod.predict(xgb.DMatrix(dftVN[features]))), axis=1)

In [64]:
np.savetxt('sub.csv'
    , subMat
    , fmt='%.5f', delimiter = ','
    , header = ','.join(['VisitNumber'] + ['TripType_' + str(lencTT.inverse_transform(i)) for i in xrange(38)]) )

In [63]:

lencTT.inverse_transform(37)

999